### Contraindicating Firmware Update

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import warnings
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

%matplotlib inline
warnings.simplefilter(action='ignore', category=UserWarning)
le = preprocessing.LabelEncoder()

#### Read Firmware Update data from csv file.

In [ ]:
firmware_data = "firmware_data_real.csv"
missing_values = ['*', '$', '-', '?', '-NA-', ',', 'n/a', 'na','NA', ' ']
columns = ["Firmware", "Version", "Firmware OS", "Device", "HardwareType", "Current Firmware Version", "Message"]
data = pd.read_csv(firmware_data, na_values=missing_values, skipinitialspace=True)
df = pd.DataFrame(data=data, columns=columns)
df.head(5)

In [ ]:
# Frequency Table formation.
firmware_data_frame = df.groupby(['Firmware', 'Device', 'Message']).size().reset_index(name="Occurance")
firmware_data_frame.head(5)

#### Naive Bayes Classifier

In [ ]:
# Adding Target attribute with Data Frame
df['Result'] = df['Message'].apply(lambda x: True if x == "Succefully update firmware" else False)
x_data_frame = df.drop(columns=['Message', 'Result'], axis=1)

# Transforming data
X = x_data_frame.apply(le.fit_transform)
y = df.apply(le.fit_transform)

# Spliting Training and Test data based on the tranformed data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4) 

# Applying train and test data into gaussian algorithm
model = GaussianNB()
model.fit(X_train, y_train.Result)
prediction = model.predict(X_test)
pred_details = {0: "Failure", 1: "Success"}
prediction_to_str = np.array([pred_details.get(i) for i in prediction])
print("Predicted Details: {0}".format(prediction_to_str))
print("")
print("Accuracy of Data: {0}".format(metrics.accuracy_score(y_test.Result, prediction)*100))

In [ ]:
# Prediction based on user's static input.
user_input_data = {
    "Firmware": ["Network_Firmware_6RGDW_WN64_09.00.03_A00-00.EXE"],
    "Version": ["09.00.03"],
    "Firmware OS": ["Windows"],
    "Device": ["PER640"],
    "HardwareType": ["Network Adatptor"],
    "Current Firmware Version": ["08.00.45"]
}

# Static transformation on data from dataframe.
transformed_user_input = {
    "Firmware": [8],
    "Version": [2],
    "Firmware OS": [2],
    "Device": [7],
    "HardwareType": [8],
    "Current Firmware Version": [1]
}

user_input_df = pd.DataFrame.from_dict(transformed_user_input)
user_input_prediction = model.predict(user_input_df)
user_prediction_to_str = np.array([pred_details.get(i) for i in user_input_prediction])
print(user_prediction_to_str)

In [ ]:
# Prediction based on user's dynamic input
user_static_input = {
    "Firmware": ["Network_Firmware_6RGDW_WN64_09.00.03_A00-00.EXE", "SAS-Drive_Firmware_R15M0_WN32_E774_A00.EXE"],
    "Version": ["09.00.03", "E774"],
    "Firmware OS": ["Windows", "Windows"],
    "Device": ["PER640", "DX6000"],
    "HardwareType": ["Network Adatptor", "SAS Drive"],
    "Current Firmware Version": ["08.00.45", "E674"]
}

# Converting user input to data frame.
user_df = pd.DataFrame.from_dict(user_static_input)

# Merging two dataframe.
merged_df = pd.concat([df, user_df], keys=["A", "B"], sort=False)
merged_df.loc["B"]

# Transformation of data frame.
trans_df = merged_df.apply(le.fit_transform)
trans_df_key = trans_df.loc['B']
trans_drop_clmn = trans_df_key.drop(columns=['Message', 'Result'])
trans_pred = model.predict(trans_drop_clmn)
trans_pred_to_str = np.array([pred_details.get(i) for i in trans_pred])
print(trans_pred_to_str)